In [ ]:
# Installing and importing dependencies
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Video Feed

# setting up video capture device
cap = cv2.VideoCapture(0)

# Making initial detections
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() # Comment this line to prevent capturing frames
        
    # Dectecting and rendering
        
        # Step #1 Recoloring image to RGB
        image = cv2. cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Grabbing the frame (feed from webcam), and recoloring/reordering it
        image.flags.writeable = False
        # setting writeable equal to False
        
        # Making Detection
        results = pose.process(image)
        
        # Resetting the writable to be attributed to True
        image.flags.writeable = True
        # Recoloring back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
    # Determining Joints
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        
        # Rendering detections via drawing utility imported from MediaPipe
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  # Changing the sensor colors
                                mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2, circle_radius = 2),
                                mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2, circle_radius = 2)
                                 )
        
        # Display image, not just frame
        cv2.imshow('Mediapipe Feed', image) # Comment this line to prevent the popup window
        
        if cv2.waitKey(10) & 0xFF == ord('q'): #0xFF is reading what we type on our keyboard
            break # Break out of our feed

cap.release()
cv2.destroyAllWindows()
# Ensure OpenCV has enough time to process the window events
for i in range(2): 
    cv2.waitKey(1)

### Determining Joints
<figure>
    <img src = "https://ai.google.dev/static/edge/mediapipe/images/solutions/pose_landmarks_index.png"
         alt = "Mediapipe Pose Detector"
         width="300" 
         height="900">
    <figcaption style="text-align: center;">Mediapipe Pose landmarker model</figcaption>
</figure>

In [ ]:
len(landmarks)

In [ ]:
landmarks = results.pose_landmarks.landmark
print("Right Wrist\n", landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
print("Right Elbow\n", landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
print("Right Shoulder\n",landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])

## Calculating Angles

In [ ]:
# Calculating Angles
def calculate_angles(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [ ]:
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
shoulder, elbow, wrist

In [ ]:
calculate_angles(shoulder, elbow, wrist)

In [ ]:
# Video Feed

# setting up video capture device
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = None

# Making initial detections
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() # Comment this line to prevent capturing frames
        
    # Dectecting and rendering
        
        # Step #1 Recoloring image to RGB
        image = cv2. cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Grabbing the frame (feed from webcam), and recoloring/reordering it
        image.flags.writeable = False
        # setting writeable equal to False
        
        # Making Detection
        results = pose.process(image)
        
        # Resetting the writable to be attributed to True
        image.flags.writeable = True
        # Recoloring back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
    # Determining Joints
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get Coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angles(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle),
                            tuple(np.multiply(elbow, [640, 280]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                 )
            
            # Pushup counter Logic
            if angle > 160:
                if stage == "down" or stage is None: 
                    stage = "up"
            
            if angle < 90:
                if stage == "up":
                    stage = "down"
                    counter += 1
                    print(counter)
            
        except:
            pass
        
        # Render curl counter
        
        # Setup status box
        cv2.rectangle(image, (0,0), (425,173), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                   (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter), (20,120), cv2.FONT_HERSHEY_SIMPLEX, 3,
                   (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (210,30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                   (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image, stage, (140,120), cv2.FONT_HERSHEY_SIMPLEX, 3,
                   (255,255,255), 2, cv2.LINE_AA)
        
        
        # Rendering detections via drawing utility imported from MediaPipe
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  # Changing the sensor colors
                                mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2, circle_radius = 2),
                                mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2, circle_radius = 2)
                                 )
        
        # Display image, not just frame
        cv2.imshow('Mediapipe Feed', image) # Comment this line to prevent the popup window
        
        if cv2.waitKey(10) & 0xFF == ord('q'): #0xFF is reading what we type on our keyboard
            break # Break out of our feed

cap.release()
cv2.destroyAllWindows()
# Ensure OpenCV has enough time to process the window events
for i in range(2): 
    cv2.waitKey(1)